In [1]:
import carla 
import math 
import time    
import random 
import numpy as np 
import cv2 
import open3d as o3d 
from matplotlib import cm 

client = carla.Client('localhost', 2000) 
client.reload_world() 
world = client.get_world() 

bp_lib = world.get_blueprint_library() 
spawn_points = world.get_map().get_spawn_points() 

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
vehicle_bp = bp_lib.find('vehicle.lincoln.mkz_2020') 
vehicle = world.try_spawn_actor(vehicle_bp, spawn_points[79]) 

spectator = world.get_spectator() 
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4, z=2.5)), vehicle.get_transform().rotation) 
spectator.set_transform(transform) 

for i in range(100): 
    vehicle_bp = random.choice(bp_lib.filter('vehicle')) 
    npc = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points)) 

for v in world.get_actors().filter('*vehicle*'): 
    v.set_autopilot(True)

In [3]:
VIRIDIS = np.array(cm.get_cmap('plasma').colors)
VID_RANGE = np.linspace(0.0, 1.0, VIRIDIS.shape[0]) 

COOL_RANGE = np.linspace(0.0, 1.0, VIRIDIS.shape[0]) 
COOL = np.array(cm.get_cmap('winter')(COOL_RANGE)) 
COOL = COOL[:, :3] 

def add_open3d_axis(vis): 
    """Add a small 3D axis on Open3D Visualizer""" 
    axis = o3d.geometry.LineSet() 
    axis.points = o3d.utility.Vector3dVector(np.array([[0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1]])) 
    axis.lines = o3d.utility.Vector2iVector(np.array([[0, 1], [0, 2], [0, 3]])) 
    axis.colors = o3d.utility.Vector3dVector(np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])) 
    vis.add_geometry(axis)

In [4]:
def lidar_callback(point_cloud, point_list): 
    """Prepares a point cloud with intensity colors ready to be consumed by Open3D """ 

    data = np.copy(np.frombuffer(point_cloud.raw_data, dtype=np.dtype('f4'))) 
    data = np.reshape(data, (int(data.shape[0] / 4), 4)) 

    # Isolate the intensity and compute a color for it 
    intensity = data[:, -1] 
    intensity_col = 1.0 - np.log(intensity) / np.log(np.exp(-0.004*100)) 
    int_color = np.c_[np.interp(intensity_col, VID_RANGE, VIRIDIS[:, 0]), 
                      np.interp(intensity_col, VID_RANGE, VIRIDIS[:, 1]), 
                      np.interp(intensity_col, VID_RANGE, VIRIDIS[:, 2])]
    
    points = data[:, :-1]  # extract the XYZ points 
    points[:, :1] = -points[:, :1]  # reverse the Y axis

    point_list.points = o3d.utility.Vector3dVector(points) 
    point_list.colors = o3d.utility.Vector3dVector(int_color) 


def radar_callback(data, point_list): 
    radar_data = np.zeros((len(data), 4)) 

    for i, detection in enumerate(data): 
        x = detection.depth * math.cos(detection.altitude) * math.cos(detection.azimuth) 
        y = detection.depth * math.cos(detection.altitude) * math.sin(detection.azimuth) 
        z = detection.depth * math.sin(detection.altitude) 

        radar_data[i, :] = [x, y, z, detection.velocity] 

    
    intensity = np.abs(radar_data[:, -1]) 
    intensity_col = 1.0 - np.log(intensity) / np.log(np.exp(-0.004*100)) 
    int_color = np.c_[np.interp(intensity_col, COOL_RANGE, COOL[:, 0]), 
                      np.interp(intensity_col, COOL_RANGE, COOL[:, 1]), 
                      np.interp(intensity_col, COOL_RANGE, COOL[:, 2])] 
    
    points = radar_data[:, :-1] 
    points[:, :1] = -points[:, :1]

    point_list.points = o3d.utility.Vector3dVector(points)
    point_list.colors = o3d.utility.Vector3dVector(int_color)

def camera_callback(image, data_dict): 
    data_dict['image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4)) 

In [5]:
lidar_bp = bp_lib.find('sensor.lidar.ray_cast') 
lidar_bp.set_attribute('range', '100.0') 
lidar_bp.set_attribute('noise_stddev', '0.1') 
lidar_bp.set_attribute('upper_fov', '15.0') 
lidar_bp.set_attribute('lower_fov', '-25.0') 
lidar_bp.set_attribute('channels', '64.0') 
lidar_bp.set_attribute('rotation_frequency', '20.0') 
lidar_bp.set_attribute('points_per_second', '500000') 

lidar_init_trans = carla.Transform(carla.Location(z=2)) 
lidar = world.spawn_actor(lidar_bp, lidar_init_trans, attach_to=vehicle) 

radar_bp = bp_lib.find('sensor.other.radar') 
radar_bp.set_attribute('horizontal_fov', '30.0') 
radar_bp.set_attribute('vertical_fov', '30.0') 
radar_bp.set_attribute('points_per_second', '10000') 
radar_init_trans = carla.Transform(carla.Location(z=2)) 
radar = world.spawn_actor(radar_bp, radar_init_trans, attach_to=vehicle) 

camera_bp = bp_lib.find('sensor.camera.rgb') 
camera_init_trans = carla.Transform(carla.Location(z=2.5, x=-3), carla.Rotation())  # Rotation(yaw=180)
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle) 

point_list = o3d.geometry.PointCloud()
radar_list = o3d.geometry.PointCloud() 

image_w = camera_bp.get_attribute('image_size_x').as_int() 
image_h = camera_bp.get_attribute('image_size_y').as_int() 

camera_data = {'imag': np.zeros((image_h, image_w, 4))} 

lidar.listen(lambda data: lidar_callback(data, point_list)) 
radar.listen(lambda data: radar_callback(data, radar_list)) 
camera.listen(lambda data: camera_callback(data, camera_data))

In [6]:
cv2.namedWindow('Camera', cv2.WINDOW_AUTOSIZE) 
cv2.imshow('RGB Camera', camera_data['image']) 
cv2.waitKey(1) 

vis = o3d.visualization.Visualizer() 
vis.create_window(
    window_name='Carla Lidar', 
    width=960, height=540, left=480, top=270) 
vis.get_render_option().background_color = [0.05, 0.05, 0.05] 
vis.get_render_option().point_size = 1 
vis.get_render_option().show_coordinate_frame = True 
add_open3d_axis(vis) 

frame = 0 

while True: 
    if frame == 2: 
        vis.add_geometry(point_list) 
        vis.add_geometry(radar_list) 
    vis.update_geometry(point_list) 
    vis.update_geometry(radar_list) 

    vis.poll_events() 
    vis.update_renderer() 

    # This is can fix Open3d jittering issues 
    time.sleep(0.005) 
    frame += 1 

    cv2.imshow('RGB Camera', camera_data['image']) 
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break 

cv2.destroyAllWindows() 
lidar.stop()
lidar.destroy()
radar.stop()
radar.destroy() 
camera.stop()
camera.destroy()
vis.destroy_window()  

for actor in world.get_spectator_actors().filter('*vechicle*'): 
    actor.destroy() 

for actor in world.get_actors().filter('*sensor*'): 
    actor.destroy()

/home/k/self-driving-repo/CARLA_0.9.13_RSS/carla_0913rss_env/lib/python3.7/site-packages/ipykernel_launcher.py:33: RuntimeWarning: divide by zero encountered in log


AttributeError: 'World' object has no attribute 'get_spectator_actors'

: 